In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import re

In [5]:
%matplotlib inline

In [6]:
# read fs_0506.csv
fs_0506 = pd.read_csv('fs0506.csv', index_col='flight_id') # May 6th flight schedule

In [7]:
# read initial_p1.csv
initial_p1 = pd.read_csv('initial_p1.csv', index_col=0) # initial pheromone matrix

In [8]:
len(initial_p1.index)

582

In [9]:
# Hyper Parameters for ant colony algorithm.
numant = 40 # 蚂蚁的个数

numcity = len(initial_p1.index) #城市的个数 582

alpha = 1  # 信息重要程度因子? 这是什么?

beta = 5 # 启发函数因子

rho = 0.1 # 信息素挥发程度

Q = 1 # 信息素常数

iter = 0 

itermax = 200

In [10]:
# pheromone matrix
pheromonetable = initial_p1.copy()

In [11]:
# Pheromone index list
# make a index list(string) corresponds to Pheromone table's index(number)
name_list = pheromonetable.index.values
# 把Pheromone表index转化为数字保存成list

In [12]:
# change Pheromone_table into array in order to be consistant as original one.
pheromonetable = pheromonetable.values

In [13]:
pathtable = np.zeros((numant, numcity)).astype(int) # 路径记录表 (每一只蚂蚁走过的城市顺序) 40x582

lengthaver = np.zeros(itermax) # cost average per iteration 200x1

lengthbest = np.zeros(itermax) # 用来记录200次迭代的最佳长度, 也就是每次的最短长度. 200x1

pathbest = np.zeros((itermax, numcity)) # 每次迭代最短路径的城市顺序 200x582

In [55]:
%pdb

Automatic pdb calling has been turned ON


In [14]:
num_plane = len(fs_0506.aircraft_id.unique()) # 121

In [15]:
# BUG: etatable is not defined
# Solution: delete etable
# BUG: name 'distmat' is not defined
# Solution: define cost function to replace distmat

In [138]:
def cancelled(translate_path):
    '''
    如果没有在tranlate_path中出现, 那么就是取消的航班.
    输入: list
    输出: list
    '''
    # translate_path 和 initial_p1 的 index 的 差.
    return list(set(initial_p1.index.values).difference(set(translate_path)))

In [139]:
def flight_air_match_dict(translate_path):
    '''
    Return two dicts refelct flight and airplane independently.
    Input: list ['axx', 'fxx', 'fxx', ... , 'axx'...]
    Output1: dict flight_airplane_dict {"fxx": "axx", ... "fxx": "axx"}
    Output2: dict airplane_flight_dict {"axx": ["fxx", ... , "fxx"], ... , "axx": ["fxx", ... , "fxx"]}
    '''
    transfered_list = ''.join(translate_path) # change list to one string.
    regex = r"(a\d+)((f\d+)+)"
    matches = re.finditer(regex, transfered_list)
    flight_airplane_dict = {} # {"fxx": "axx", ... "fxx": "axx"}
    airplane_flight_dict = {} # {"axx": ["fxx", ... , "fxx"], ... , "axx": ["fxx", ... , "fxx"]}
    for matchNum, match in enumerate(matches):
        a = match.group(1)
        f = match.group(2)
        regex = r"f\d+"
        f = re.findall(regex, f)
        flight_airplane_dict = merge_two_dicts(flight_airplane_dict, {i: a for i in f})
        airplane_flight_dict = merge_two_dicts(airplane_flight_dict, {a : f})
    return flight_airplane_dict, airplane_flight_dict

In [140]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def calculate_x(flight_airplane_dict, cancell_list):
    '''
    calculate x from flight_air_dict and cancell_list
    if cancelled , x = 0
    if new path is not consistent with old one, means airplane has been changed.
    input
    -----
    flight_airplane_dict: {"fxx": "axx", ... "fxx": "axx"}
    cancell_list: ["fxx", "fxx", ..., "fxx"]
    return
    ------
    x_dict: {"fxx": bool, ...}
    '''
    
    keys = flight_airplane_dict.keys()
    x_dict = {key: int(flight_airplane_dict[key] != fs_0506.loc[key].aircraft_id) for key in keys}
    x_dict = merge_two_dicts(x_dict, {cancelled_flight_id: 0 for cancelled_flight_id in cancell_list})
    return x_dict

In [141]:
# debug cancelled return []
# solutiuon: change position of diff.

In [142]:
# cost function
def calculate_xyz(path):
    """calculate x y z for cost funtion"""
    translate_path = [name_list[i] for i in path]
    cancelled_list = cancelled(translate_path)
    print(cancelled_list)
    # print("cancell list length", len(cancelled_list))
    fad, afd = flight_air_match_dict(translate_path) # {flight:air} and {air:flight}
    x_dict = calculate_x(fad, cancelled_list)
    return x_dict
    

# def assign_func(translate_path):
#     '''
#     Input: list
#     step1: turn list of string into one string.
#     step2: find flight in time sequence.
#     Return: {'a19':['f2033', ...,'f1963']}
#     '''
#     # step1
#     transfered_list = ''.join(translate_path) # change list to one string.
#     x_dict = calculate_x(transfered_list)  # {flight_id: 1, ...., flight_id:0}
#     return x_dict
def cost(path):
    """return a matrix"""
    # cost is a combination of x, y, z, indicate flight status.
    calculate_xyz(path)

In [197]:
# test cost fucntion
_test_path = np.random.permutation(range(1, len(initial_p1.index[:-100])))
_test_path = np.insert(_test_path, 0, 0)
print(_test_path)
print("test path length is ", len(_test_path))
# test translate_path
calculate_xyz(_test_path)
# test assign_func step 1
# assign_func(calculate_xyz(_test_path))
# test cancelled function
len(calculate_xyz(_test_path))

[  0 195 206  47  59 318 249 117  80 354 381 334 312 289 145 353 244  81
  91 279 154 196 464 258 247 308 161  12 345 281 388 323 316 329 280   7
 359   5 178 111 423 371 449 224 223 298 374  43  13 400 204 331  38 106
 240  28   1 335 452 356 339 456 313 214 311 166 476 474 168  90 412 309
 264 232 162 446 433 272  31  64 197 330  21 156 137 383 260  74   3 209
 437 347 440 171 109 393 175 288 403   9 337 150 333 413  69 158  37 186
 180 245 188  48 415  67 406 155 445 302 165 265 133 163  77  82 434 119
 200 226  60 361 236 248   8 105  61 322 306 422 387 376 126 398 454 409
 447 243  23  54 414 229 468 370 231 235 212 299 340 416 254 438 253 256
 108 364  20 349 293 443 131  34 140 251 107 130 193 473 179 250  50  17
 475 373 436 276 360 129 397 153  18 386 113  56  99 255  32  68  87 396
 296 421 160  70 358  15 191 187 189  73 466 481 173 170  19 246 228 455
  46 325 273  78 319 266 220 417 332 216 385 177  51 407 367 448  85 124
 310 275 215 350 305 287 450 362  84 199 425  52 35

461

In [65]:
while iter < itermax:
    # 随机产生蚂蚁的起点城市
    if numant <= num_plane:
        pathtable[:, 0] = np.random.permutation(range(0, num_plane))[:numant] # 把起始城市按照蚂蚁数量随机分配
    else: # 对于蚂蚁比城市多的情况, 也是把所有城市随机分给每一个蚂蚁
        pathtable[:num_plane, 0] = np.random.permutation(range(0, num_plane))[:] # 先分配前面的蚂蚁, 保证每个城市都可能作为起点
        pathtable[num_plane:, 0] = np.random.permutation(range(0, num_plane))[:numant - num_plane] # 再分配多出的蚂蚁, 随机分配
    length = np.zeros(numant) # 蚂蚁的路径距离
    
    for i in range(numant): # 对于每一只蚂蚁
        # pathtable 40 x 52
        visiting = pathtable[i, 0] # 起始城市的位置, 就是之前随机分配的城市. visiting = 
        # visited = set() # 使用set记录所有去过的城市, 因为set是集合, 集合中的元素不可以重复.
        # visited.add(visiting) # 每去一个城市, 把当前城市添加到集合里面.
        unvisited = set(range(numcity)) # 创建为访问的城市集合
        unvisited.remove(visiting) # 删除访问过的城市, 留下的就是没有访问的
        
        for j in range(1, numcity): # 访问完所有的城市需要的步数
            # 下面是轮盘法的代码, 使用轮盘法选择下一个城市
            # 轮盘法可能是一个选取城市的一个公式.
            listunvistited = list(unvisited)
            probtrans = np.zeros(len(listunvistited))
            
            for k in range(len(listunvistited)):
                # 这个下面就是那个很复杂的方程.t时刻,蚂蚁k从城市i到城市j的概率的分子
                # probtrans[k] = np.power(pheromonetable[visiting][listunvistited[k]], alpha)*np.power(etatable[visiting][listunvistited[k]], beta)
                probtrans[k] = np.power(pheromonetable[visiting][listunvistited[k]], alpha)
                # @TODO: 这里他写的是alpha, 按照公式应该是beta.
            # 计算概率
            cumsumprobtrans = (probtrans/sum(probtrans)).cumsum()
            # 概率随机减去一点
            cumsumprobtrans -= np.random.rand() # 减去的这一点可以让某些概率为负值.
            k = listunvistited[np.where(cumsumprobtrans>0)[0][0]] # 下一个访问的城市. 城市为概率大于0的第一个城市.
            # @TODO: 这个地方可能会有bug, 评论里面说的
            pathtable[i,j] = k # 记录第i只蚂蚁在第j步访问的城市.
            unvisited.remove(k) # 移除访问过的城市
            # visited.add(k)
            
            #length[i] += distmat[visiting][k] # 把蚂蚁之前城市到目前城市的距离算到总路程.
            visiting = k # 目前城市定位到k
            length[i] = cost(path, )
        length[i] += distmat[visiting][pathtable[i, 0]] # 从目前的城市回归起点城市的距离.
        # print("第{}蚂蚁总共走了{}米".format(i, length[i]))
    #set_trace()
    # 包含所有蚂蚁的一个迭代结束后, 统计这次迭代的统计参数.
    lengthaver[iter] = length.mean() # 所有的蚂蚁在这一个循环中走过的平均距离
    if iter == 0:
        lengthbest[iter] = length.min()
        pathbest[iter] = pathtable[length.argmin()].copy() # 走过路径最短的那个蚂蚁的路线作为最佳路线
    else:
        if length.min() > lengthbest[iter-1]: # 如果当前的最短路径大于上一次迭代的最短路径
            lengthbest[iter] = lengthbest[iter-1] # 最短的路径依然是上次迭代的结果
            pathbest[iter] = pathbest[iter-1].copy() # 最佳路径依然是之前的路径
        else: # 如果当前路径最佳,更新最佳路径和最佳距离
            lengthbest[iter] = length.min()
            pathbest[iter] = pathtable[length.argmin()].copy()
            print("最短路径{}米".format(lengthbest[iter]))
            print("经过城市{}".format(pathbest[iter]))
    # 更新信息素
    change_pheromone_table = np.zeros((numcity, numcity))
    for i in range(numant):
        for j in range(numcity-1):
            change_pheromone_table[pathtable[i,j]][pathtable[i, j+1]] += Q/distmat[pathtable[i,j]][pathtable[i, j+1]]
        change_pheromone_table[pathtable[i, j+1]][pathtable[i, 0]] += Q/distmat[pathtable[i, j+1]][pathtable[i, 0]] # 从最后一个城市回到起点
    pheromonetable = (1-rho)*pheromonetable + change_pheromone_table
    
    iter += 1
    
    if (iter-1)%20 == 0:
        print("当前迭代次数{}".format(iter-1))        

NameError: name 'distmat' is not defined